In [24]:
youtube_template_string = """
You are an assistant in football (soccer) scouting, and provides answers to questions by using fact based information.
Use the following information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer from the context, just say that you don't know

<context>
{context}
</context>

<question>
{question}
</question>

{format_instruction}
Do this format for every unique player id.
"""

In [25]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

player_id_schema = ResponseSchema(name="player_id",
                                  description="ID of the player that the report is about which is an integer between 0 and 99999999")

report_summary_schema = ResponseSchema(name="report_summary",
                                  description="Summary of the report content about the player")

response_schemas = [player_id_schema, report_summary_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instruction = output_parser.get_format_instructions()

print(format_instruction)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Milvus


EMBEDDING_MODEL = "nomic-embed-text"
MODEL = "mistral"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

COLLECTION_NAME = "scouting"
VECTOR_STORE_URI = "http://localhost:19530"


connection_args = {'uri': VECTOR_STORE_URI}
vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    vector_field="embeddings",
    primary_field="id",
    auto_id=True
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

##### Dont have to edit anything below this to change models
def format_docs(docs):
    returnString = "\n\n".join(f"Player ID: {doc.metadata['player_transfermarkt_id']}, Report-Content: " + doc.page_content for doc in docs)
    print(returnString)
    return returnString


In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=youtube_template_string)

query = "I need a central defender who's very secure defensively"

retrieved_documents = retriever.invoke(query)

formatted_docs = format_docs(retrieved_documents)

messages = prompt.format_messages(context=formatted_docs, question=query, format_instruction=format_instruction)

messages

In [ ]:
print(messages[0].content)

In [ ]:
response = model.invoke(messages)

response